# Hvor mange barn kommenterer på norske TikTok-byggeres profiler?

Dokumentasjon av metode  
*Christian Nicolai Bjørke, 7.11.2022*

In [2]:
import pandas as pd
import sqlite3

# Koble til database
db = "/PATH/brukere.db"
conn = sqlite3.connect(db)

Databasen består av følgende kolonner: 
* poster (navn på TikTok-brukeren som har laget videoen)
* poster_username (URL til TikTok-brukeren som har laget videoen)
* post_text (tekst som hører til TikTok-videoen)
* name (navn på TikTok-brukeren som har kommentert på videoen)
* username (URL til TikTok-brukeren som har kommentert på videoen))
* comment (kommentaren)

### 1. Samle data

Først skraper vi alle kommentarene på sidene til 50 av de største TikTok-byggere i Norge. Alt blir samlet i en database.  

Størrelsen på databasen:

In [1]:
query = """SELECT * FROM Brukere"""
comments = len(pd.read_sql_query(query, conn))
print(f"Databasen består av {comments} kommentarer")

### 2. Finne barna

Deretter finner vi alle kommentarene der noen oppgir å være 9-14 eller 15-17 år.

Lage to nye kolonner:

ALTER TABLE Brukere
ADD barn9_14 TEXT

ALTER TABLE Brukere
ADD barn15_17 TEXT

#### 9-14 år
*SQL-kode for å hente ut ulike måter å skrive alderen på. Må etterpå gås igjennom manuelt for å luke ut feil.*

UPDATE Brukere
SET barn9_14 = "1"
WHERE (comment LIKE "% 9 år%" or comment like "% 10 år%" or comment like "% 11 år%" or comment like "% 12 år%" 
        or comment like "% 13 år%" or comment like "% 14 år%"
        or comment like "% 10år%" or comment like "% 11år%" or comment like "% 12år%" 
        or comment like "% 13år%" or comment like "% 14år%"
        or comment like "%ni år%" or comment like "%ti år%" or comment like "%elleve år%" 
        or comment like "%tolv år%" or comment like "%tretten år%" or comment like "%fjorten år%" 
        or comment like "%niår%" or comment like "%tiår%" or comment like "%elleveår%" 
        or comment like "%tolvår%" or comment like "%trettenår%" or comment like "%fjortenår%")
        AND username NOT IN 
        (SELECT poster_username FROM Brukere)

#### 15-17 år
*SQL-kode for å hente ut ulike måter å skrive alderen på. Må etterpå gås igjennom manuelt for å luke ut feil.*

UPDATE Brukere
SET barn15_17 = "1"
WHERE (comment LIKE "% 15 år%" or comment like "% 16 år%" or comment like "% 17 år%"
        or comment like "% 15år%" or comment like "% 16år%" or comment like "% 17år%"
        or comment like "%femten år%" or comment like "%seksten år%" or comment like "%sytten år%" 
        or comment like "%femtenår%" or comment like "%sekstenår%" or comment like "%syttenår%" 
        or comment like "%søtten år%" or comment like "%søttenår%"
        AND username NOT IN 
        (SELECT poster_username FROM Brukere)

### 3. Gå igjennom manuelt

Alle treffene blir gått igjennom manuelt for å luke ut kommentarer som viser til noe annet enn alder, eller som helt tydelig er tull. 

### 4. Telle opp antall barn
Vi finner brukernavnene på de kommentarene som er igjen, og grupperer dem slik at hvert brukernavn bare kommer opp én gang.

In [ ]:
query9_14 = """SELECT * FROM Brukere
                WHERE barn9_14 = "1"
                GROUP BY username;"""

query15_17 = """SELECT * FROM Brukere
                WHERE barn15_17 = "1"
                GROUP BY username;"""

df9_14 = pd.read_sql_query(query9_14, conn)
usernames9_14 = len(df9_14)
df15_17 = pd.read_sql_query(query15_17, conn)
usernames15_17 = len(df15_17)

print(f"{usernames9_14} brukere oppgir alder mellom 9 og 14 år.")
print(f"{usernames15_17} brukere oppgir alder mellom 15 og 17 år.")

### 5. Trekke ut kommentarer fra brukernavn

Når vi har brukernavnene, kan vi samle alle kommentarer disse brukerne har lagt igjen på muskelmennenes sider. 

Først lager vi et eget table med brukernavnene.

CREATE TABLE "Brukernavn9_14" AS
SELECT username FROM Brukere
WHERE barn9_14 = "1"
GROUP BY username;

CREATE TABLE "Brukernavn15_17" AS
SELECT username FROM Brukere
WHERE barn15_17 = "1"
GROUP BY username;

Deretter bruker vi inner join for å koble brukernavn med kommentarer.

In [ ]:
comments9_14 = """SELECT Brukere.poster, Brukere.username, Brukere.comment FROM Brukere
                    INNER JOIN Brukernavn9_14 ON Brukernavn9_14.username = Brukere.username;"""

comments15_17 = """SELECT Brukere.poster, Brukere.username, Brukere.comment FROM Brukere
                    INNER JOIN Brukernavn15_17 ON Brukernavn15_17.username = Brukere.username;"""

df_comments9_14 = pd.read_sql_query(comments9_14, conn)
df_comments15_17 = pd.read_sql_query(comments15_17, conn)

print(f"{len(df_comments9_14)} kommentarer er skrevet av barn som oppgir å være mellom 9 og 14 år.")
print(f"{len(df_comments15_17)} kommentarer er skrevet av barn som oppgir å være mellom 15 og 17 år.")

### 6. Hvilke byggere får flest kommentarer fra barn?

In [ ]:
top_comments_9_14 = """SELECT poster, count(*) FROM Kommentarer9_14
                        GROUP BY poster
                        ORDER BY count(*) DESC"""

df = pd.read_sql_query(top_comments_9_14, conn)

print("Disse byggerne får flest kommentarer fra 9-14-åringer:")
df.head(5)

In [ ]:
top_comments_15_17 = """SELECT poster, count(*) FROM Kommentarer15_17
                        GROUP BY poster
                        ORDER BY count(*) DESC"""

df = pd.read_sql_query(top_comments_15_17, conn)

print("Disse byggerne får flest kommentarer fra 15-17-åringer:")
df.head(5)

### 7. Hvor mange visninger har norske byggere til sammen?

In [ ]:
total_views = """SELECT SUM(views) FROM Brukerviews"""

print("Visninger for 50 norske TikTok-byggere:")
print(pd.read_sql_query(total_views, conn))

### 8. Hvor mange visninger har den enkelte bygger fått?

In [ ]:
builder_views = """SELECT poster, SUM(views) FROM Brukerviews
                GROUP BY poster
                ORDER BY SUM(views) DESC"""

df_builder_views = pd.read_sql_query(builder_views, conn)
df_builder_views.head(5)


### 9. Hvor mange brukere har lagt igjen kommentar?

In [ ]:
query_users = """SELECT username FROM Brukere
                GROUP BY username;"""

df_users = pd.read_sql_query(query_users, conn)
usernames_total = len(df_users)

print(f"{usernames_total} brukere har lagt igjen en kommentar.")